In [55]:
import requests
import pandas as pd

# Your API key for FoodData Central
api_key = 'uvc2RJDlyvc4xJFsFp7v9REGVi4Uq26il1UCEmXD'

# Base URL for FoodData Central
base_url = "https://api.nal.usda.gov/fdc/v1/foods/search"

# Foods to search for
foods = {"Lentils": 172420, 
         "Broccoli":2346366, 
         "Cauliflower":2346374, 
         "Mushrooms":2346370, 
         "Sweet Potato":169304,
         "Ginger": 169231,
         "Tumeric":172231,
         "Tahini":2343062,
         "Cider vinegar":173469,
         "Hemp Seeds":170148, 
         "Pumpkin Seeds":170556,
         "Olive Oil":171413,
         
         "Walnuts":170187, 
         "Macadamia":170178, 
         "Brazil nuts":170569,
         "Almonds":170567,
         "Sunflower seeds":170562,
         "Blueberries":2344771, 
         "Chia seeds":170554,
         "Flax seeds":2343063,
         "Pomegranate juice":167787,
         "Soy joghurt":2342919,
         "Cocoa powder":170657
         }

food_amounts = {"Lentils": 70, 
                "Broccoli":250, 
                "Cauliflower":100, 
                "Mushrooms":50, 
                "Sweet Potato":100,
                "Ginger": 3,
                "Tumeric":2,
                "Tahini":15,
                "Cider vinegar":10,
                "Hemp Seeds":20, 
                "Pumpkin Seeds":10,
                "Olive Oil":25,
                
                "Walnuts":25, 
                "Macadamia":20, 
                "Brazil nuts":7,
                "Almonds":10,
                "Sunflower seeds":10,
                "Blueberries":90, 
                "Chia seeds":20,
                "Flax seeds":4,
                "Pomegranate juice":50,
                "Soy joghurt":75,
                "Cocoa powder":20
                }

# Nutrients of interest
nutrients_of_interest = {
    "Engery": 1008,
    "Total Fat": 1004,
    "Saturated Fat": 1258,
    "Monounsaturated Fat": 1292,
    "Polyunsaturated Fat": 1293,
    "Cholesterol": 1253,
    "Carbohydrate": 1005,
    "Fiber": 1079,
    "Sugars": 2000,
    "Protein": 1003,
    
    "Calcium": 1087,
    "Iron": 1089,
    "Magnesium": 1090,
    "Phosphorus": 1091,
    "Potassium": 1092,
    "Sodium": 1093,
    "Zinc": 1095,
    "Copper": 1098,
    "Manganese": 1101,
    "Selenium": 1103,
    
    "Vitamin A": 1106,
    "Thiamin (B1)": 1165,
    "Riboflavin (B2)": 1166,
    "Niacin (B3)": 1167,
    "Pantothenic Acid (B5)": 1170,
    "Vitamin B6": 1175,
    "Folate (B9)": 1177,
    "Vitamin B12": 1178,
    "Vitamin C": 1162,
    "Vitamin D": 1110,
    "Vitamin E": 1109,
    "Vitamin K": 1185,
    
    "Choline": 1180,
}

In [52]:
# Function to get nutrient data for a given food ID
def get_nutrient_data(food_id, nutrients):
    url = f"https://api.nal.usda.gov/fdc/v1/food/{food_id}"
    params = {'api_key': api_key}
    response = requests.get(url, params=params).json()

    nutrient_data = {}
    for nutrient in nutrients:
        nutrient_data[nutrient] = None  # Default to None if nutrient not found
        if 'foodNutrients' in response:
            for item in response['foodNutrients']:
                if 'nutrient' in item and item['nutrient']['id'] == nutrients[nutrient]:
                    nutrient_data[nutrient] = item.get('amount')
                    break

    return nutrient_data


In [53]:
# Main script to compile nutrient data for each food
nutrient_table = pd.DataFrame(index=foods.keys(), columns=nutrients_of_interest.keys())
for name, id in foods.items():
    nutrients = get_nutrient_data(id, nutrients_of_interest)
    nutrient_table.loc[name] = nutrients
    
# scale every item to food amount
for name, amount in food_amounts.items():
    nutrient_table.loc[name] = nutrient_table.loc[name] * amount / 100

nutrient_table["Amount"] = food_amounts.values()

In [60]:
nutrient_table

,Engery,Total Fat,Saturated Fat,Monounsaturated Fat,Polyunsaturated Fat,Cholesterol,Carbohydrate,Fiber,Sugars,Protein,...,Pantothenic Acid (B5),Vitamin B6,Folate (B9),Vitamin B12,Vitamin C,Vitamin D,Vitamin E,Vitamin K,Choline,Amount
Lentils,246.4,0.742,0.1078,0.1351,0.3682,0.0,44.345,7.49,1.421,17.241,...,1.498,0.378,335.3,0.0,3.15,0.0,0.343,3.5,67.48,70
Broccoli,102.5,0.875,0.1025,0.0275,0.045,0.0,16.325,6.25,3.65,6.7,...,NaN,0.4475,145.0,0.0,202.0,NaN,0.4,265.0,48.75,250
Cauliflower,26.0,0.29,0.135,0.035,0.032,0.0,5.18,2.1,1.99,2.0,...,NaN,0.172,50.0,0.0,42.7,NaN,0.08,16.1,46.1,100
Mushrooms,19.5,0.215,0.031,0.0,0.1,0.0,2.58,0.6,1.24,1.81,...,NaN,0.046,18.5,0.025,1.1,NaN,0.005,0.0,10.8,50
Sweet Potato,35.0,0.34,0.065,0.012,0.134,0.0,7.38,1.9,5.48,2.18,...,0.2,0.16,49.0,0.0,1.5,0.0,0.96,108.6,21.0,100
Ginger,2.4,0.0225,0.00609,0.00462,0.00462,0.0,0.5331,0.06,0.051,0.0546,...,0.00609,0.0048,0.33,0.0,0.15,0.0,0.0078,0.003,0.864,3
Tumeric,6.24,0.065,0.03676,0.00898,0.01512,0.0,1.3428,0.454,0.0642,0.1936,...,0.01084,0.00214,0.4,0.0,0.014,0.0,0.0886,0.268,0.984,2
Tahini,89.25,8.07,1.1295,3.045,3.54,0.0,3.18,1.395,0.0735,2.55,...,NaN,0.02235,14.7,0.0,0.0,NaN,0.0375,0.0,3.87,15
Cider vinegar,2.1,0.0,0.0,0.0,0.0,0.0,0.093,0.0,0.04,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
Hemp Seeds,110.6,9.75,0.92,1.08,7.62,0.0,1.734,0.8,0.3,6.312,...,NaN,0.12,22.0,NaN,0.1,NaN,0.16,NaN,NaN,20


In [61]:
nutrient_table.sum()

Engery                    1744.18
Total Fat                 111.216
Saturated Fat            16.37118
Monounsaturated Fat      47.55113
Polyunsaturated Fat      41.01162
Cholesterol                   0.0
Carbohydrate             143.9992
Fiber                      45.061
Sugars                    32.1043
Protein                   63.1271
Calcium                    640.19
Iron                      20.8763
Magnesium                   744.5
Phosphorus                1768.73
Potassium                 3737.85
Sodium                     204.34
Zinc                       12.696
Copper                    3.71604
Manganese                 6.37648
Selenium                  190.326
Vitamin A                   173.0
Thiamin (B1)              2.27015
Riboflavin (B2)           1.56436
Niacin (B3)               15.4447
Pantothenic Acid (B5)     2.39951
Vitamin B6                1.85663
Folate (B9)                740.05
Vitamin B12                 0.025
Vitamin C                 256.177
Vitamin D     